In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import keras
import csv
import string
from string import digits
import re
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding , LSTM , Dense
from keras.models import Model
from keras.callbacks import EarlyStopping as early, ModelCheckpoint as checkpoint  

Using TensorFlow backend.


In [0]:
!ls

drive  sample_data


In [0]:
data = pd.read_csv('drive/My Drive/Colab Notebooks/Machine_Translation/kan-eng/kan.txt', sep="\t", header=None)
data.columns = ["eng","kan","created_by"]

In [5]:
data.head()

,eng,kan,created_by
0,Tom woke up.,ಟಾಮ್ ಏಳಿದನು.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Give me half.,ಅರ್ಧ ನನಗೆ ಕೊಡು.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,We needed it.,ನಮಗೆ ಬೇಕಾಗಿತ್ತು.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
3,Tom liked you.,ಟಾಮ್ ನಿನ್ನನ್ನು ಬಯಸಿದನು.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Just go inside.,ಒಳಗೆ ಹೋಗು ಅಷ್ಟೇ.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [0]:
def preprocess(df):
  """
  Preprocessing function to convert data into a standard form.
  Since dataset is pretty much in standard format, little pre-processing is required.
  Will also add start and end token to target data
  <SOS> : will imply start of the sentence
  <EOS> : will imply end of the sentence
  """
  df["eng"] = df["eng"].apply(lambda x: x.lower()) # Converting to lower case
  special_ch = set(string.punctuation) #Removing all special characters, if any
  df["eng"] = df["eng"].apply(lambda x: ''.join(ch for ch in x if ch not in special_ch))
  df["kan"] = df["kan"].apply(lambda x: x.replace('.',''))
  remove_digits = str.maketrans('', '', digits)
  df["eng"] = df["eng"].apply(lambda x: x.translate(remove_digits))
  df["kan"] = df["kan"].apply(lambda x : '<SOS> '+ x + ' <EOS>')
  
  return df
  

In [0]:
data = preprocess(data).copy()

In [8]:
data.head()

,eng,kan,created_by
0,tom woke up,<SOS> ಟಾಮ್ ಏಳಿದನು <EOS>,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,give me half,<SOS> ಅರ್ಧ ನನಗೆ ಕೊಡು <EOS>,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,we needed it,<SOS> ನಮಗೆ ಬೇಕಾಗಿತ್ತು <EOS>,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
3,tom liked you,<SOS> ಟಾಮ್ ನಿನ್ನನ್ನು ಬಯಸಿದನು <EOS>,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,just go inside,<SOS> ಒಳಗೆ ಹೋಗು ಅಷ್ಟೇ <EOS>,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [9]:
data.shape

(149, 3)

In [0]:
eng_vocab = set()
for line in data['eng']:
  for word in line.split():
    if word not in eng_vocab:
      eng_vocab.add(word)

kan_vocab=set()
for kan in data.kan:
    for word in kan.split():
        if word not in kan_vocab:
            kan_vocab.add(word)

### Identifying maximum length of sequences in source and destination:

len_eng = []
for line in data['eng']:
  len_eng.append(len(line.split(' ')))

max_len_src = np.max(len_eng)

len_kan = []
for line in data['kan']:
  len_kan.append(len(line.split(' ')))

max_len_tar = np.max(len_kan)



In [11]:
print("Maximum length of sentence in source language: ",max_len_src )
print("Maximum length of sentence in source language: ",max_len_tar )

Maximum length of sentence in source language:  15
Maximum length of sentence in source language:  11


In [12]:
input_words = sorted(list(eng_vocab))
target_words = sorted(list(kan_vocab))
print("Input words: \n", input_words)
print("Output Words: \n", target_words)
num_encoder_tokens = len(eng_vocab)
num_decoder_tokens = len(kan_vocab)
num_decoder_tokens += 1 

input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

Input words: 
 ['a', 'about', 'abroad', 'accident', 'across', 'afraid', 'after', 'ago', 'agreement', 'agrees', 'all', 'almost', 'always', 'among', 'an', 'and', 'animals', 'announced', 'another', 'any', 'anybody', 'anything', 'apology', 'are', 'argentina', 'arrived', 'as', 'ask', 'at', 'awake', 'away', 'baby', 'back', 'be', 'beach', 'because', 'been', 'before', 'believe', 'believed', 'better', 'book', 'bored', 'borrow', 'both', 'bottles', 'bought', 'brazil', 'brother', 'brought', 'bucket', 'but', 'buy', 'by', 'call', 'came', 'can', 'cant', 'car', 'careless', 'caught', 'cause', 'caused', 'cellar', 'charge', 'children', 'coffee', 'competitor', 'concert', 'conditions', 'cost', 'could', 'couldnt', 'couple', 'crowd', 'crush', 'dangerous', 'daughter', 'day', 'deal', 'death', 'decided', 'declared', 'did', 'didnt', 'difficult', 'do', 'does', 'doesnt', 'doing', 'dollars', 'dont', 'down', 'dress', 'drink', 'during', 'earlier', 'eat', 'eaten', 'else', 'engagement', 'enough', 'even', 'ever', 'every

In [0]:
X,y = data['eng'], data['kan']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.1) 

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 8):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_len_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_len_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_len_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [0]:
latent_dim = 25

encoder_inputs = Input(shape = (None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim, mask_zero= True)(encoder_inputs)
enc_lstm = LSTM(latent_dim, return_state= True)
output , h , c = enc_lstm(encoder_embedding)

encoder_states = [h,c]

decoder_inputs = Input(shape=(None,))
decoder_emb = Embedding(num_decoder_tokens, latent_dim, mask_zero= True)
dec_emb = decoder_emb(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences= True, return_state= True)
decoder_output, _, _ = decoder_lstm(dec_emb,initial_state = encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_output = decoder_dense(decoder_output)

model = Model([encoder_inputs, decoder_inputs], decoder_output)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 8
epochs = 50

In [18]:
print(train_samples)
print(val_samples)

134
15


In [19]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
16/16 [==============================] - 4s 220ms/step - loss: 3.1748 - acc: 0.1584 - val_loss: 2.8586 - val_acc: 0.1724
Epoch 2/50
16/16 [==============================] - 1s 65ms/step - loss: 3.0247 - acc: 0.1780 - val_loss: 2.7483 - val_acc: 0.1724
Epoch 3/50
16/16 [==============================] - 1s 66ms/step - loss: 2.8866 - acc: 0.1752 - val_loss: 2.6912 - val_acc: 0.1724
Epoch 4/50
16/16 [==============================] - 1s 64ms/step - loss: 2.7504 - acc: 0.1767 - val_loss: 2.6928 - val_acc: 0.1724
Epoch 5/50
16/16 [==============================] - 1s 63ms/step - loss: 2.7125 - acc: 0.1755 - val_loss: 2.7172 - val_acc: 0.1724
Epoch 6/50
16/16 [==============================] - 1s 67ms/step - loss: 2.6633 - acc: 0.1772 - val_loss: 2.7510 - val_acc: 0.1724
Epoch 7/50
16/16 [==============================] - 1s 65ms/step - loss: 2.6776 - acc: 0.1752 - val_loss: 2.7895 - val_acc: 0.1724
Epoch 8/50
16/16 [==============================] - 1s 63ms/step - loss: 2.6674 - 

In [0]:
model.save_weights('drive/My Drive/Colab Notebooks/Machine_Translation/kan-eng/mt_weights.h5')

In [0]:
model.load_weights('drive/My Drive/Colab Notebooks/Machine_Translation/kan-eng/mt_weights.h5')

In [0]:

encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= decoder_emb(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [0]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['<SOS>']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        if (sampled_char == '<EOS>' or
           len(decoded_sentence) > 50):
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [33]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Kannada Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Kannada Translation:', decoded_sentence[:-4])

Input English sentence: he had an accident at work
Actual Kannada Translation: ಕೆಲಸದಲ್ಲಿ ಅವನಿಗೆ ಅಪಘಾತವಾಯಿತು <
Predicted Kannada Translation:  ನಾನು ಬೇಕಿತ್ತೋ ಅಂತ ಹಂದಿಮನೆ <
